In [1]:
import numpy as np
from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.table import Column
from numpy import *
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.wcs import WCS
from astropy.io import fits
from matplotlib.colors import LogNorm
from astropy.utils.data import download_file
import warnings

catalog_path='/Users/EvanB.LAPTOP-8CHCIOMA/Downloads/'

#A100-NSA Catalog
myTab=Table.read(catalog_path+'a100-nsa-for-adap.fits')

In [2]:
# R-band Magnitude Limit
Mrmax = -18.2
Mr = myTab['ABSMAG'][:,4]
Mrflag = Mr < Mrmax

# Velocity input
vinput = int(input())
v = myTab['Vhelio']
vflag = (v > -1*vinput) & (v < vinput)

cut = Mrflag & vflag

cuttab = myTab[cut]

 1500


In [3]:
print(len(myTab['ABSMAG'][:,4]))
print(len(cuttab['ABSMAG'][:,4]))

39098
10692


In [4]:
#cuttab.info

# Computing 5th Nearest Neighbor

In [5]:
#Computes 5 nearest neighbors of one selected galaxy

# Defining ra and dec, making empty array for fifth nearest neighbors for later use
ra = np.array(cuttab['RA'])
dec = np.array(cuttab['DEC'])
fifthnn = np.zeros(len(ra))

# Sets up array of tuples of ra and dec points for each galaxy
points = []
for i in range(len(dec)):
    points.append((ra[i],dec[i]))

def distance(ra1, dec1, ra2, dec2):
    return ((ra1 - ra2) ** 2 + (dec1 - dec2) ** 2) ** ( 1 / 2)

def kClosest(points, target, K):
    pts = []
    n = len(points)
    d = []
 
    for i in range(n):
        d.append({
            "first": distance(points[i][0], points[i][1], target[0], target[1]), 
            "second": i
        })
     
    d = sorted(d, key=lambda l:l["first"])
 
    for i in range(K):
        pt = []
        pt.append(points[d[i]["second"]][0])
        pt.append(points[d[i]["second"]][1])
        pts.append(pt)
 
    return pts

#Initial parameters
target = points[0]
K = 6

#Computes 5 nearest neighbors of one selected galaxy
dist = []
fnn = []

for pt in kClosest(points, target, K):
    dist.append(distance(pt[0], pt[1], target[0], target[1]))
    fnn.append((pt[0],pt[1]))
    #print(f"{pt[0]} {pt[1]}")
    
print(f"Closest Points = {fnn[1:5]}")

print(f"Distances (Degrees) = {dist}")

Closest Points = [(0.6580024759606975, 16.643731319970442), (0.8674337580835826, 16.73615246179746), (1.591291431871461, 17.437278420621006), (1.581731326757165, 14.327454416164155)]
Distances (Degrees) = [0.0, 0.5214704238783213, 0.592989329927536, 1.508153366598167, 1.974232935138005, 2.5430251209849937]


In [22]:
#print(points)

In [47]:
#Computes 5 nearest neighbors of each galaxy

# Defining ra and dec, making empty array for fifth nearest neighbors for later use
ra = np.array(cuttab['RA'])
dec = np.array(cuttab['DEC'])
fifthnn = np.zeros(len(ra))

# Sets up array of tuples of ra and dec points for each galaxy
points = []
for i in range(len(dec)):
    points.append((ra[i],dec[i]))

def distance(ra1, dec1, ra2, dec2):
    return ((ra1 - ra2)**2 + (dec1 - dec2)**2)**(1/2)

def kClosest(points, target, K):
    pts = []
    n = len(points)
    d = []
 
    for i in range(n):
        d.append({
            "first": distance(points[i][0], points[i][1], target[0], target[1]), 
            "second": i
        })
     
    d = sorted(d, key=lambda l:l["first"])
 
    for i in range(K):
        pt = []
        pt.append(points[d[i]["second"]][0])
        pt.append(points[d[i]["second"]][1])
        pts.append(pt)
 
    return pts

#Initial parameters
K = 6

#Computes 5 nearest neighbors of one selected galaxy
dist = []
fnn = []
for i in range(len(points)):
    target = points[i]
    for pt in kClosest(points, target, K):
        dist.append(distance(pt[0], pt[1], target[0], target[1]))
        fnn.append((pt[0],pt[1]))
        #print(f"{pt[0]} {pt[1]}")
    
#print(f"Closest Points = {fnn[1:5]}")

#print(f"Distances (Degrees) = {dist}")

In [48]:
sublist = [dist[n:n+6] for n in range(0, len(dist), 6)]
print(sublist[0:2])

[[0.0, 0.5214704238783213, 0.592989329927536, 1.508153366598167, 1.974232935138005, 2.5430251209849937], [0.0, 0.9008014396663964, 1.1802036211171796, 2.528267123887345, 2.634988284098588, 2.644761089426649]]


In [39]:
print(dist[0:24])

[0.0, 0.5214704238783213, 0.592989329927536, 1.508153366598167, 1.974232935138005, 2.5430251209849937, 0.0, 0.9008014396663964, 1.1802036211171796, 2.528267123887345, 2.634988284098588, 2.644761089426649, 0.0, 0.7318585292694487, 0.7581749907905204, 0.9240213314685154, 1.0851890088997718, 1.1010926719538061, 0.0, 2.0345159882890553, 2.0767847899867653, 2.55448298501058, 2.662269064998005, 2.8926550654735537]


In [29]:
print(len(points))
print(len(dist))

10692
64152
